# Data Processing

In [4]:
from google.colab import drive,output
drive.mount('/content/drive')

In [5]:
!pip install catboost
!pip install optuna
!pip install category_encoders
!pip install imbalanced-learn  # imblearn이 설치되어 있지 않은 경우 설치
output.clear()


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [6]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from catboost import CatBoostClassifier, Pool

from category_encoders import CatBoostEncoder
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import MultiLabelBinarizer

import warnings
warnings.filterwarnings("ignore")  # Optional: Suppress warnings

In [7]:
path = '/Users/seoeunseo/Desktop/DACON/제출파일/'

In [8]:
train_df = pd.read_csv(path+"final_train5.csv",index_col=0)
test_df = pd.read_csv(path+"final_test5.csv",index_col=0)

# project_id 관련 컬럼 추출
project_id_columns = [col for col in train_df.columns if 'project_id' in col]

# 모델링

In [9]:
# Step 1: X와 y 데이터 준비
# 'WT'와 각 돌연변이를 이진화하여 모델 입력으로 변환
mlb = MultiLabelBinarizer()
X = mlb.fit_transform(train_df[project_id_columns].apply(lambda x: x.values.tolist(), axis=1))

# 각 행의 most_frequent_non_wt 값을 y로 설정
y = train_df['SUBCLASS']

# 레이블 인코딩
le = LabelEncoder()
y = le.fit_transform(y)

# 카테고리 인코딩
encoder = CatBoostEncoder()
X_train_encoded = encoder.fit_transform(train_df.drop(project_id_columns + ['SUBCLASS', 'ID'], axis=1), y)

# 최종 학습 데이터 생성
train_X = pd.concat([X_train_encoded, pd.DataFrame(X)], axis=1)
train_X.columns = train_X.columns.astype(str)

In [10]:
# SMOTEENN 적용
smote_enn = SMOTEENN(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(train_X, y)

# 리샘플링된 데이터 확인 (선택 사항)
print(f"Original dataset shape: {train_X.shape}")
print(f"Resampled dataset shape: {X_resampled.shape}")

Original dataset shape: (6201, 4411)
Resampled dataset shape: (15971, 4411)


In [11]:
# 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# 설정
random_state = 42
n_trials = 10  # 최적화를 더 잘 수행하기 위해 n_trials를 증가시킴

# 클래스 가중치 계산
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = [weights[i] for i in range(len(classes))]  # 리스트 형태로 변환

# CatBoost용 Pool 객체 생성 (범주형 피처 지정이 필요하면 cat_features 인수 추가)
train_pool_full = Pool(X_train, y_train)
valid_pool = Pool(X_valid, y_valid)

# CatBoost

In [12]:
# 최적의 하이퍼파라미터로 최종 모델 학습
final_model = CatBoostClassifier(
    iterations=687,
    learning_rate=0.07969454818643935,
    depth=7,
    l2_leaf_reg=6.387926357773329,
    random_strength=2.4041677639819286,
    bagging_temperature=0.15599452033620265,
    one_hot_max_size=2,
    class_weights=class_weights,
    loss_function='MultiClass',
    task_type='GPU',
    devices='0',
    verbose=0,
    random_seed=random_state
)

final_model.fit(
    train_pool_full,
    eval_set=valid_pool,
    early_stopping_rounds=50,
    use_best_model=True,
    verbose=False
)

# 예측 및 평가
y_pred_encoded = final_model.predict(valid_pool)

# Macro F1-Score 계산
macro_f1 = f1_score(y_valid, y_pred_encoded, average='macro')
print(f'Macro F1-Score: {macro_f1:.4f}')

# 예측값을 원래 SUBCLASS로 복원
y_pred = le.inverse_transform(y_pred_encoded.astype(int))

# 실제 값도 원래 SUBCLASS로 복원
y_valid_original = le.inverse_transform(y_valid)

# 예측과 실제 값의 일부 출력
print(f'Original Predicted Classes: {y_pred[:10]}')
print(f'Original True Classes: {y_valid_original[:10]}')

CatBoostError: catboost/libs/train_lib/trainer_env.cpp:9: Environment for task type [GPU] not found

In [ ]:
class CancerClassifier:
    def __init__(self, model, encoder, le, mlb):
        self.model = model
        self.encoder = encoder
        self.le = le
        self.mlb = mlb

    def predict(self, test_df):
        # 원본 데이터 수정 방지를 위해 복사
        test_df = test_df.copy()

        # 'ID' 열 제거
        if 'ID' in test_df.columns:
            test_X = test_df.drop(columns=['ID'])
        else:
            test_X = test_df.copy()

        # 'project_id'로 시작하는 컬럼 식별
        project_id_columns = [col for col in test_X.columns if 'project_id' in col]

        # 'SUBCLASS' 열 초기화
        test_df['SUBCLASS'] = None

        # 1단계: 'project_id'를 사용하여 'SUBCLASS' 할당
        if project_id_columns:
            # 'WT'가 아닌 가장 빈도가 높은 값을 'SUBCLASS'로 할당하는 함수
            def most_frequent_non_wt(row):
                non_wt_values = row[row != 'WT']
                if non_wt_values.empty:
                    return None
                mode_value = non_wt_values.mode()[0]  # 최빈값
                count = (non_wt_values == mode_value).sum()  # 최빈값의 개수
                return mode_value if count > 6 else None  # 최빈값이 6를 넘으면 할당, 그렇지 않으면 None

            # 각 행에 함수 적용
            test_df['SUBCLASS'] = test_df[project_id_columns].apply(most_frequent_non_wt, axis=1)

        # 5개 이하로 예측된 값들이 None으로 설정된 행들 식별
        missing_indices = test_df['SUBCLASS'].isna()

        # 2단계: AI 모델을 사용하여 'SUBCLASS' 예측
        if missing_indices.any():
            # 모델을 위한 데이터 준비
            test_X_missing = test_X.loc[missing_indices]

            # 'project_id' 컬럼 이진화 (학습 시 사용한 mlb를 사용)
            X_project_id = self.mlb.transform(test_X_missing[project_id_columns].apply(lambda x: x.values.tolist(), axis=1))
            X_project_id = pd.DataFrame(X_project_id, index=test_X_missing.index)

            # 나머지 컬럼들 선택 (project_id 컬럼 제외)
            test_X_missing_other = test_X_missing.drop(columns=project_id_columns)

            # 범주형 변수 인코딩
            test_X_missing_encoded = self.encoder.transform(test_X_missing_other)

            # 최종 입력 데이터 생성
            test_X_missing_final = pd.concat([test_X_missing_encoded.reset_index(drop=True), X_project_id.reset_index(drop=True)], axis=1)
            test_X_missing_final.columns = test_X_missing_final.columns.astype(str)

            # 모델을 사용하여 예측
            y_test_pred_encoded = self.model.predict(test_X_missing_final)

            # 예측 결과를 원래 라벨로 변환
            original_labels = self.le.inverse_transform(y_test_pred_encoded.astype(int))

            # 예측된 'SUBCLASS'를 누락된 인덱스에 할당
            test_df.loc[missing_indices, 'SUBCLASS'] = original_labels

        # 최종 'SUBCLASS' 예측 반환
        return test_df['SUBCLASS']

In [ ]:
# 학습된 모델과 전처리 도구들을 사용하여 분류기 초기화
classifier = CancerClassifier(
    model=final_model,
    encoder=encoder,
    le=le,
    mlb=mlb
)

# Submission

In [ ]:
# 예측 수행
test_df['SUBCLASS'] = classifier.predict(test_df)

# 제출 파일 생성 및 저장
submission_path = path+"sample_submission.csv"
submission = pd.read_csv(submission_path)
submission['SUBCLASS'] = test_df['SUBCLASS']

In [ ]:
submission['SUBCLASS'].value_counts()

In [ ]:
# 제출 파일 저장
output_path = path+"sample_submission_code_1020_ver1(threshold6).csv"
submission.to_csv(output_path, index=False)

print(f'Submission file saved to {output_path}')